In [23]:
import tweepy
from datetime import timedelta

# Get Tweets

In [1]:
consumer_key = 'XXXXXXXXX'
consumer_secret = 'XXXXXXXXX'
access_token = 'XXXXXXXXX'
access_token_secret= 'XXXXXXXXX'

In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [29]:

tweets = []
dates =[{"start" : "2019-04-18", "end":"2019-04-19"},
        {"start" : "2019-04-19", "end":"2019-04-20"},
        {"start" : "2019-04-20", "end":"2019-04-21"},
        {"start" : "2019-04-21", "end":"2019-04-22"},
        {"start" : "2019-04-22", "end":"2019-04-23"}]

for each_date in dates:
    print(each_date)
    for status in tweepy.Cursor(api.search,q="#ibm",
                                since=each_date["start"], until=each_date["end"],tweet_mode='extended',
                               result_type='recent', include_entities=True, monitor_rate_limit=True, 
                               wait_on_rate_limit=True,  lang="en").items():
        tweets.append(status)

{'start': '2019-04-18', 'end': '2019-04-19'}
{'start': '2019-04-19', 'end': '2019-04-20'}
{'start': '2019-04-20', 'end': '2019-04-21'}
{'start': '2019-04-21', 'end': '2019-04-22'}
{'start': '2019-04-22', 'end': '2019-04-23'}


In [32]:
len(tweets)

1842

In [33]:
tweets_text = [i.full_text for i in tweets]

In [34]:
tweets_text[:4]

['“Equality is not about fixing women, it’s about fixing everything else!” -Larry Fink #womensbondclub #blackrock #ibm https://t.co/62v8gt8sAN',
 'RT @cloudbetweter: Great blog by @andy_gower from @IBMcloud categorized as #Infrastructure (strange). Leading to an #IBM commissioned study…',
 'RT @TDASUK_FrankR: Arm Yourself with @IBMCloud Strategy - The world is changing rapidly. Invest in your talent and skills. Start here to la…',
 'RT @unixbhaskar: Okay, showing off :) my #fedora system and this fellow is rolling over from build to build.\n\n@nixcraft @harishpillay @kern…']

# Get Saved Tweets

In [35]:
with open('tweets_file.txt','r') as f:
    tweets_from_file = f.read()
    
listof_tweets_from_file = tweets_from_file.split('|||')
listof_tweets_from_file = listof_tweets_from_file[:-1]
print(len(listof_tweets_from_file))
listof_tweets_from_file[:4]

1898


['“Equality is not about fixing women, it’s about fixing everything else!” -Larry Fink #womensbondclub #blackrock #ibm https://t.co/62v8gt8sAN',
 'RT @cloudbetweter: Great blog by @andy_gower from @IBMcloud categorized as #Infrastructure (strange). Leading to an #IBM commissioned study…',
 'RT @TDASUK_FrankR: Arm Yourself with @IBMCloud Strategy - The world is changing rapidly. Invest in your talent and skills. Start here to la…',
 'RT @unixbhaskar: Okay, showing off :) my #fedora system and this fellow is rolling over from build to build.\n\n@nixcraft @harishpillay @kern…']

# Prepare the data for LDA

In [30]:
#source: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
stemmer = PorterStemmer()

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [42]:
processed_docs = []
for i in listof_tweets_from_file:
    processed_docs.append(preprocess(i))

dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# LDA on Bag of Words Approach ( Term Frequencies or Count Vectorizer)

In [45]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}\n'.format(idx, topic))

Topic: 0 
Words: 0.022*"cybersecur" + 0.019*"learn" + 0.019*"secur" + 0.019*"need" + 0.018*"build" + 0.018*"bigdata" + 0.017*"busi" + 0.017*"trust" + 0.016*"solut" + 0.016*"help"

Topic: 1 
Words: 0.046*"blockchain" + 0.032*"encrypt" + 0.019*"time" + 0.018*"amazon" + 0.017*"ibmz" + 0.017*"thomas_harr" + 0.017*"turn" + 0.016*"cybersecur" + 0.016*"cyberresili" + 0.015*"boost"

Topic: 2 
Words: 0.118*"ibmsystem" + 0.031*"socialcto" + 0.025*"proudibm" + 0.020*"ibmz" + 0.017*"berlin" + 0.017*"free" + 0.017*"opportun" + 0.016*"brand" + 0.016*"miss" + 0.016*"present"

Topic: 3 
Words: 0.028*"world" + 0.022*"secur" + 0.022*"watson" + 0.021*"ibmcloud" + 0.021*"join" + 0.015*"employe" + 0.014*"blockchain" + 0.014*"manag" + 0.014*"hide" + 0.013*"rhsummit"

Topic: 4 
Words: 0.032*"week" + 0.031*"client" + 0.030*"help" + 0.025*"develop" + 0.024*"announc" + 0.022*"learn" + 0.019*"cloud" + 0.018*"global" + 0.017*"innov" + 0.017*"thrill"

Topic: 5 
Words: 0.034*"blockchain" + 0.034*"join" + 0.029*"clo

### Topic Identification

    Based on the keywords, we can identify the following topics:
     
    Topic: CYBERSECURITY
    Words:,cybersecur,learn,secur,need,build,bigdata,busi,trust,solut,help

    Topic: BLOCKCHAIN SECURITY
    Words:,blockchain,encrypt,time,amazon,ibmz,thomas_harr,turn,cybersecur,cyberresili,boost

    Topic: IBM BRAND
    Words:,ibmsystem,socialcto,proudibm,ibmz,berlin,free,opportun,brand,miss,present

    Topic: WATSON CLOUD
    Words:,world,secur,watson,ibmcloud,join,employe,blockchain,manag,hide,rhsummit

    Topic: CLIENT RELATIONS
    Words:,week,client,help,develop,announc,learn,cloud,global,innov,thrill

    Topic: BLOCKCHAIN CLOUD
    Words:,blockchain,join,cloud,partner,chain,suppli,help,summit,platform,infrastructur

    Topic: THINK AT BERLIN
    Words:,berlin,ibmsystem,thinkatibm,thomas_harr,data,ibmtechu,proudibm,socialcto,meet,think

    Topic: BLOCKCHAIN WATSON TEAM
    Words:,blockchain,watson,team,certif,busi,solut,data,demand,cognit,profession

    Topic: -- 
    Words:,thomas_harr,nodexl,learn,topcybernew,cloud,spectrum,cyber,come,virtual,ibmstorag

    Topic: BLOCKCHAIN MARKET
    Words:,blockchain,chain,suppli,market,work,watson,data,enterpris,analyt,track

    

# LDA on TFIDF

In [46]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)


for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWord: {}\n'.format(idx, topic))

Topic: 0 
Word: 0.022*"secur" + 0.016*"cybersecur" + 0.016*"today" + 0.016*"world" + 0.014*"news" + 0.013*"iotexchang" + 0.013*"watson" + 0.013*"work" + 0.013*"microsoft" + 0.012*"servic"

Topic: 1 
Word: 0.031*"berlin" + 0.030*"topcybernew" + 0.029*"chidambara" + 0.028*"hide" + 0.025*"nodexl" + 0.024*"thinkatibm" + 0.022*"thomas_harr" + 0.020*"summit" + 0.019*"infrastructur" + 0.018*"ibmtechu"

Topic: 2 
Word: 0.021*"blockchain" + 0.018*"data" + 0.015*"want" + 0.015*"power" + 0.015*"client" + 0.014*"network" + 0.014*"announc" + 0.014*"video" + 0.012*"cloud" + 0.012*"prior"

Topic: 3 
Word: 0.033*"ibmsystem" + 0.024*"proudibm" + 0.018*"experi" + 0.017*"socialcto" + 0.016*"turn" + 0.014*"chevron" + 0.014*"applic" + 0.014*"netflix" + 0.013*"generalmotor" + 0.013*"amazon"

Topic: 4 
Word: 0.041*"blockchain" + 0.028*"volkswagen" + 0.025*"chain" + 0.025*"suppli" + 0.020*"build" + 0.017*"great" + 0.016*"univers" + 0.016*"week" + 0.016*"skill" + 0.015*"miner"

Topic: 5 
Word: 0.026*"manag" + 

    Based on the keywords, we can identify the following topics:
    
    Topic 0: CYBER SECURITY
    Word:,secur,cybersecur,today,world,news,iotexchang,watson,work,microsoft,servic,

    Topic 1: THINK AT BERLIN
    Word:,berlin,topcybernew,chidambara,hide,nodexl,thinkatibm,thomas_harr,summit,infrastructur,ibmtechu,

    Topic 2: BLOCKCHAIN IN DATA AND NETWORKS
    Word:,blockchain,data,want,power,client,network,announc,video,cloud,prior,

    Topic 3: IBM SOCIAL
    Word:,ibmsystem,proudibm,experi,socialcto,turn,chevron,applic,netflix,generalmotor,amazon,

    Topic 4: BLOCKCHAIN IN SUPPLY CHAIN
    Word:,blockchain,volkswagen,chain,suppli,build,great,univers,week,skill,miner,

    Topic 5: DATA SOLUTIONS
    Word:,manag,data,team,talk,certif,check,note,best,solut,avail,

    Topic 6: IBM CLOUD
    Word:,ibmsystem,cloud,compani,rhsummit,blockchain,miss,brand,work,interact,opportun,

    Topic 7: INNOVATION AT IBM
    Word:,innov,understand,decad,mikequindazzi,loveibm,help,enterpris,ilikeither,advanc,rometti,

    Topic 8: MARKETS
    Word:,market,encrypt,thomas_harr,futur,share,sale,drug,discoveri,watson,ibmsystem,

    Topic 9: WATSON SYSTEMS
    Word:,watson,game,come,ahead,davidgokhshtein,blockchaintech,ibmsystem,thinkatibm,meet,ibmstorag,


# Classification of an Unseen Text (Tweet)

Using LDA with TF

In [96]:
unseen_document = 'IBM signs banking hybrid cloud services deal with Boursorama'
print(unseen_document)

IBM signs banking hybrid cloud services deal with Boursorama


In [47]:

bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\nTopic: {}\n".format(score, lda_model.print_topic(index, 5)))

Score: 0.7749557495117188
Topic: 0.028*"world" + 0.022*"secur" + 0.022*"watson" + 0.021*"ibmcloud" + 0.021*"join"

Score: 0.025011036545038223
Topic: 0.047*"thomas_harr" + 0.040*"nodexl" + 0.034*"learn" + 0.034*"topcybernew" + 0.031*"cloud"

Score: 0.025008708238601685
Topic: 0.034*"blockchain" + 0.034*"join" + 0.029*"cloud" + 0.027*"partner" + 0.022*"chain"

Score: 0.025006350129842758
Topic: 0.022*"cybersecur" + 0.019*"learn" + 0.019*"secur" + 0.019*"need" + 0.018*"build"

Score: 0.025006070733070374
Topic: 0.032*"week" + 0.031*"client" + 0.030*"help" + 0.025*"develop" + 0.024*"announc"

Score: 0.02500336430966854
Topic: 0.043*"blockchain" + 0.030*"watson" + 0.018*"team" + 0.018*"certif" + 0.017*"busi"

Score: 0.02500297874212265
Topic: 0.063*"berlin" + 0.058*"ibmsystem" + 0.058*"thinkatibm" + 0.039*"thomas_harr" + 0.032*"data"

Score: 0.025002775713801384
Topic: 0.045*"blockchain" + 0.029*"chain" + 0.028*"suppli" + 0.025*"market" + 0.024*"work"

Score: 0.025001661852002144
Topic: 0.

Using LDA with TFIDF

In [48]:
unseen_document = 'IBM signs banking hybrid cloud services deal with Boursorama'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\nTopic: {}\n".format(score, lda_model.print_topic(index, 5)))

Score: 0.774971604347229
Topic: 0.022*"cybersecur" + 0.019*"learn" + 0.019*"secur" + 0.019*"need" + 0.018*"build"

Score: 0.025007976219058037
Topic: 0.063*"berlin" + 0.058*"ibmsystem" + 0.058*"thinkatibm" + 0.039*"thomas_harr" + 0.032*"data"

Score: 0.025005392730236053
Topic: 0.043*"blockchain" + 0.030*"watson" + 0.018*"team" + 0.018*"certif" + 0.017*"busi"

Score: 0.02500409632921219
Topic: 0.118*"ibmsystem" + 0.031*"socialcto" + 0.025*"proudibm" + 0.020*"ibmz" + 0.017*"berlin"

Score: 0.025002971291542053
Topic: 0.045*"blockchain" + 0.029*"chain" + 0.028*"suppli" + 0.025*"market" + 0.024*"work"

Score: 0.025002378970384598
Topic: 0.034*"blockchain" + 0.034*"join" + 0.029*"cloud" + 0.027*"partner" + 0.022*"chain"

Score: 0.025001803413033485
Topic: 0.028*"world" + 0.022*"secur" + 0.022*"watson" + 0.021*"ibmcloud" + 0.021*"join"

Score: 0.02500159852206707
Topic: 0.046*"blockchain" + 0.032*"encrypt" + 0.019*"time" + 0.018*"amazon" + 0.017*"ibmz"

Score: 0.025001170113682747
Topic: 0.